In [1]:
import glob
import os
import warnings

import pandas as pd
from gplately.tools import plate_isotherm_depth

from lib.coregister_combined_point_data import run_coregister_combined_point_data
from lib.coregister_crustal_thickness import run_coregister_crustal_thickness
from lib.coregister_magnetic import coregister_magnetic
from lib.misc import (
    calculate_slab_flux,
    calculate_water_thickness,
)
from lib.pu import generate_grid_points
from lib.slab_dip import calculate_slab_dip

# Suppress occasional joblib warnings
%env PYTHONWARNINGS=ignore::UserWarning

warnings.simplefilter("ignore", UserWarning)

env: PYTHONWARNINGS=ignore::UserWarning


In [2]:
n_jobs = 8
times = range(171)
verbose = False

grid_resolution = 0.5  # degrees

In [3]:
model_dir = "EarthBytePlateMotionModel-ARCHIVE/Global_Model_WD_Internal_Release_2019_v2_Clennett_NE_Pacific"

data_dir = "data"
agegrid_dir = os.path.join(data_dir, "AgeGrids")
sedthick_dir = os.path.join(data_dir, "SedimentThickness")
carbonate_dir = os.path.join(data_dir, "CarbonateThickness")
co2_dir = os.path.join(data_dir, "CrustalCO2")
crustal_thickness_dir = os.path.join(data_dir, "CrustalThickness")
mag_anomaly_filename = os.path.join(
    data_dir,
    "MagneticAnomaly",
    "emag2_upcont_interpolated.nc",
)
subducted_quantities_dir = os.path.join(
    data_dir,
    "SubductedQuantities",
)

output_dir = "outputs"
subduction_data_filename = os.path.join(output_dir, "subducting_plate_data.csv")
study_area_dir = os.path.join(output_dir, "study_area_polygons")
output_filename = os.path.join(output_dir, "grid_data.csv.zip")

feature_filenames = glob.glob(
    os.path.join(
        model_dir,
        "*.gpml",
    )
)
rotation_filenames = glob.glob(
    os.path.join(
        model_dir,
        "*.rot",
    )
)

### Generate grid points

In [4]:
grid_data = generate_grid_points(
    times=times,
    resolution=grid_resolution,
    polygons_dir=study_area_dir,
    topological_features=feature_filenames,
    rotation_model=rotation_filenames,
    n_jobs=n_jobs,
    verbose=verbose,
)

### Load subduction data

In [5]:
subduction_data = pd.read_csv(subduction_data_filename)
subduction_data["plate_thickness (m)"] = plate_isotherm_depth(
    subduction_data["seafloor_age (Ma)"],
    maxiter=100,
)
subduction_data = calculate_water_thickness(subduction_data)
subduction_data = calculate_slab_flux(subduction_data)
subduction_data = calculate_slab_dip(subduction_data)

### Assign subduction data to grid

In [6]:
coregistered_data = run_coregister_combined_point_data(
    point_data=grid_data,
    subduction_data=subduction_data,
    n_jobs=n_jobs,
    verbose=verbose,
    output_filename=None,
)

### Assign magnetic anomaly and crustal thickness data to grid

In [7]:
coregistered_data = run_coregister_crustal_thickness(
    point_data=coregistered_data,
    input_dir=crustal_thickness_dir,
    n_jobs=n_jobs,
    verbose=verbose,
)
coregistered_data = coregister_magnetic(
    data=coregistered_data,
    filename=mag_anomaly_filename,
    n_jobs=n_jobs,
)
coregistered_data.to_csv(output_filename, index=False)